In [ ]:

import sqlite3
import pandas as pd

# get the data from the database
def extract_data(sqlite_file, model_name, visualize=False):
    conn = sqlite3.connect(sqlite_file)
    nvtx_df = pd.read_sql_query("SELECT start, end, textId FROM NVTX_EVENTS", conn)
    stringID_df = pd.read_sql_query("SELECT * FROM StringIds", conn)
    conn.close()
    
    nvtx_df['duration(ms)'] = ((nvtx_df['end'] - nvtx_df['start']) / 1000000) # (ns -> ms)
    
    nvtx_df = pd.merge(nvtx_df, stringID_df, left_on='textId', right_on='id', how='left')
    nvtx_df = nvtx_df.drop(['textId', 'id', 'start', 'end'], axis=1)
    nvtx_df = nvtx_df.rename(columns={'value': 'function_name'})
    
    functionList = ["3D_backbone", "bbox_head", "find_centers","post_processing", "reader", "transformer"]
    nvtx_df = nvtx_df[nvtx_df['function_name'].isin(functionList)]
    nvtx_df = nvtx_df.groupby(['function_name']).mean()
    
    if visualize:
        title = model_name + ' profiling result(%)'
        nvtx_df.plot.pie(y='duration(ms)', figsize=(12, 5), title=title, autopct='%1.1f%%', xlabel='', ylabel='', legend=False)
    
    # total duration을 구함
    total_duration = nvtx_df['duration(ms)'].sum()
    nvtx_df.loc['total_duration'] = total_duration
    nvtx_df['model_name'] = model_name
    
    return nvtx_df


In [ ]:
result_dir = 'results/time'
file_names = ['baseline', 'poolformer', 'mem_opt', 'trt_32', 'trt_16', 'backbone_opt']

dataframes = []
for file_name in file_names:
    sqlite_file = f'{result_dir}/{file_name}.sqlite'
    model_df = extract_data(sqlite_file, file_name, visualize=False)
    dataframes.append(model_df)

combined_df = pd.concat(dataframes)
combined_df = combined_df.reset_index()
combined_df = combined_df.pivot(index='model_name', columns='function_name', values='duration(ms)')

# column reordering
column_order = ['reader', '3D_backbone', 'find_centers', 'transformer', 'bbox_head', 'post_processing', 'total_duration']
combined_df = combined_df.reindex(column_order, axis=1)

row_order = ['baseline', 'poolformer', 'mem_opt', 'trt_32', 'trt_16', 'backbone_opt']
combined_df = combined_df.reindex(row_order)

In [ ]:
combined_df